In [12]:
import tensorflow as tf
from veggideas.transfer import load_non_trainable_model
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, layers
from keras import models
from keras import optimizers
from veggideas.load_data import load_train_data, load_val_data

In [13]:
train_data = load_train_data()
val_data = load_val_data()
base_model = load_non_trainable_model()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [14]:
small_val_data = val_data.take(5)
small_train_data = train_data.take(5)

In [ ]:
def data_augmentation():
    datagen = ImageDataGenerator(
        rotation_range=20,  # Rotate images by 20 degrees
        width_shift_range=0.2,  # Shift images horizontally by 20% of the total width
        height_shift_range=0.2,  # Shift images vertically by 20% of the total height
        shear_range=0.2,  # Apply shear transformation with a shear intensity of 20%
        zoom_range=0.2,  # Apply zoom transformation with a zoom range of 20%
        horizontal_flip=True,  # Flip images horizontally
        fill_mode='nearest'  # Fill newly created pixels during transformations with the nearest value
    )

    # Create an empty list to store augmented images and labels
    augmented_images = []
    augmented_labels = []

    # Iterate over each batch in the training dataset
    for images, labels in train_data:
        # Apply data augmentation to the batch
        augmented_batch = datagen.flow(images, labels, batch_size=images.shape[0])
        augmented_images.extend(augmented_batch[0][0])
        augmented_labels.extend(augmented_batch[0][1])

    # Convert the augmented images and labels to TensorFlow tensors
    augmented_images = tf.convert_to_tensor(augmented_images)
    augmented_labels = tf.convert_to_tensor(augmented_labels)

    # Create a new dataset using the augmented images and labels
    augmented_train_data = tf.data.Dataset.from_tensor_slices((augmented_images, augmented_labels))

    return augmented_train_data

In [ ]:
###making the bounding box

In [11]:
def add_last_layers(base_model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    base_model = base_model
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1))
    dense_layer_2 = layers.Dense(64, activation='relu')
    reg_l2 = regularizers.L2(0.001)
    dense_layer_reg = layers.Dense(128, activation='relu', bias_regularizer=reg_l2)
    conv2D_256 = layers.Conv2D(512, 3, padding='same', activation='relu')
    conv2D_512 = layers.Conv2D(512, 3, padding='same', activation='relu')
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    prediction_layer = layers.Dense(15, activation='softmax')

    model = models.Sequential([
        resize_and_rescale,
        base_model,
        conv2D_256,
        maxpool_layer,
        conv2D_512,
        maxpool_layer,
        flattening_layer,
        dense_layer,
        dense_layer_reg,
        dense_layer_2,
        prediction_layer
        ])


    opt = optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(base_model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)

Epoch 1/10
5/5 [==============================] - 20s 4s/step - loss: 2.7298 - accuracy: 0.0375 - val_loss: 2.7388 - val_accuracy: 0.0812
Epoch 2/10
5/5 [==============================] - 21s 5s/step - loss: 2.6902 - accuracy: 0.0750 - val_loss: 2.7147 - val_accuracy: 0.0500
Epoch 3/10
5/5 [==============================] - 22s 5s/step - loss: 2.6805 - accuracy: 0.0812 - val_loss: 2.6762 - val_accuracy: 0.0875
Epoch 4/10
5/5 [==============================] - 22s 5s/step - loss: 2.6570 - accuracy: 0.1063 - val_loss: 2.6626 - val_accuracy: 0.0938
Epoch 5/10
5/5 [==============================] - 21s 5s/step - loss: 2.6353 - accuracy: 0.1000 - val_loss: 2.6446 - val_accuracy: 0.1125
Epoch 6/10
5/5 [==============================] - 22s 5s/step - loss: 2.6127 - accuracy: 0.2062 - val_loss: 2.6504 - val_accuracy: 0.1375
Epoch 7/10
5/5 [==============================] - 21s 5s/step - loss: 2.5975 - accuracy: 0.2250 - val_loss: 2.6515 - val_accuracy: 0.1187
Epoch 8/10
5/5 [==================

In [24]:
import numpy as np 
# Resize images to the required input size of the object detection model
resized_images = tf.image.resize(train_data, (255, 255))

# Convert resized images to numpy arrays
image_arrays = resized_images.numpy()

# Run object detection on the batch of images
detections = model_transfer(image_arrays)

# Extract bounding boxes from the detections
batch_bounding_boxes = []
for detection in detections:
    num_detections = int(detection['num_detections'])
    boxes = detection['detection_boxes'][:num_detections]
    batch_bounding_boxes.append(boxes)

batch_bounding_boxes = np.array(batch_bounding_boxes)

ValueError: Attempt to convert a value (<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>) to a Tensor.